Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [85]:
%pwd

'C:\\Users\\malmt\\OneDrive\\Desktop\\Lambda\\DS-Unit-4-Sprint-1-NLP\\module3-document-classification'

In [86]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import neighbors, datasets

import pandas as pd



In [87]:
# You may need to change the path
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [88]:
train.shape

(4087, 3)

In [89]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [90]:
x_train = train["description"]
y_train = train["ratingCategory"]

### Define Pipeline Components

In [91]:
# Creation of Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', rfc)
                ])
# The pipeline puts together a bunch fit then transform,fit then predict. 

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [92]:
%pwd

'C:\\Users\\malmt\\OneDrive\\Desktop\\Lambda\\DS-Unit-4-Sprint-1-NLP\\module3-document-classification'

In [93]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=2, n_jobs=-1, verbose=1)
grid_search.fit(x_train, y_train)

Fitting 2 folds for each of 32 candidates, totalling 64 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done  64 out of  64 | elapsed:   21.3s finished


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('clf', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'clf__max_depth': (15, 20),
                         'clf__n_estimators': (5, 10),
                         'vect__max_df': (0.75, 1.0),
                         'vect__max_features': (500, 1000),
                         'vect__min_df': (0.02, 0.05)},
             verbose=1)

In [103]:
grid_search.best_score_

0.7036943068129133

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [95]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [96]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [97]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [98]:
subNumber = 0

In [99]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'C:/Users/malmt/OneDrive/Desktop/Lambda/DS-Unit-4-Sprint-1-NLP/module3-document-classification/data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [100]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=50, 
                   algorithm='randomized',
                   n_iter=40)

In [101]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
lsi = Pipeline([('vect', vect), ('svd', svd)])
rfc = RandomForestClassifier()

pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [102]:
parameters = { 
    'lsi__svd__n_components': [75,100,125],
    'lsi__vect__max_df':[1,2,3,4,5],
    'lsi__vect__min_df': [1,2,3],
    'lsi__vect__max_features': [50, 200],
    'clf__n_estimators':[50,100],
    'clf__max_depth':[15,100]
}

grid_search = GridSearchCV(pipe,parameters, cv=3, n_jobs=-1, verbose=3)
grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 360 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   43.7s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:  8.5min finished


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('lsi',
                                        Pipeline(steps=[('vect',
                                                         TfidfVectorizer(ngram_range=(1,
                                                                                      2),
                                                                         stop_words='english')),
                                                        ('svd',
                                                         TruncatedSVD(n_components=50,
                                                                      n_iter=40))])),
                                       ('clf', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'clf__max_depth': [15, 100],
                         'clf__n_estimators': [50, 100],
                         'lsi__svd__n_components': [75, 100, 125],
                         'lsi__vect__max_df': [1, 2, 3, 4, 5],
               

### Make a Submission File

In [104]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [105]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [106]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [107]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'C:/Users/malmt/OneDrive/Desktop/Lambda/DS-Unit-4-Sprint-1-NLP/module3-document-classification/data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

In [108]:
print(grid_search.best_score_)
grid_search.best_params_

0.7036943068129133


{'clf__max_depth': 100,
 'clf__n_estimators': 100,
 'lsi__svd__n_components': 125,
 'lsi__vect__max_df': 1,
 'lsi__vect__max_features': 200,
 'lsi__vect__min_df': 1}

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [116]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint



In [117]:
%pwd

'C:\\Users\\malmt\\OneDrive\\Desktop\\Lambda\\DS-Unit-4-Sprint-1-NLP\\module3-document-classification'

In [118]:
# Continue Word Embedding Work Here
import spacy
nlp = spacy.load("en_core_web_lg")

In [119]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [120]:
X = get_word_vectors(train['description'])
len(X) == len(train['description'])

True

In [121]:
X_test = get_word_vectors(test['description'])

In [123]:
rfc.fit(X, train['ratingCategory'])

RandomForestClassifier()

In [124]:
rfc.score(X, train['ratingCategory'])

1.0

In [125]:
rfc.predict(X_test)

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [126]:
test['ratingCategory'] = rfc.predict(X_test)

In [127]:
test[['id', 'ratingCategory']].to_csv('testSolutionSubmission.csv', header=True, index=False)

### Make a Submission File

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?